# Validating measures

In [106]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import time

### Data Directory

In [107]:
INPUT_DIR = '../data/interim/'
OUTPUT_DIR = '../data/processed/'

### Reading predicted file

In [108]:
ip_file = INPUT_DIR + 'dbscan_results_3000_0.001.csv'
df_pred = pd.read_csv(ip_file, sep=',')

In [109]:
df_pred.head()

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas13,meas14,meas15,meas16,meas17,meas20,meas21,unit,time_cycles,labels
0,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,...,2388.02,8138.62,8.4195,0.03,392,39.06,23.4190,1,1,0
1,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,...,2388.07,8131.49,8.4318,0.03,392,39.00,23.4236,1,2,0
2,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,...,2388.03,8133.23,8.4178,0.03,390,38.95,23.3442,1,3,0
3,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,...,2388.08,8133.83,8.3682,0.03,392,38.88,23.3739,1,4,0
4,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,...,2388.04,8133.80,8.4294,0.03,393,38.90,23.4044,1,5,0


In [110]:
# Make everything in labels column into 0 or -1
df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)

### Generating Ground Truth with heuristics

In [111]:
timecycles_for_failure = 10

In [112]:
df_pred['ground_truth'] = 0

In [113]:
df_pred.head()

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas14,meas15,meas16,meas17,meas20,meas21,unit,time_cycles,labels,ground_truth
0,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,...,8138.62,8.4195,0.03,392,39.06,23.4190,1,1,0,0
1,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,...,8131.49,8.4318,0.03,392,39.00,23.4236,1,2,0,0
2,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,...,8133.23,8.4178,0.03,390,38.95,23.3442,1,3,0,0
3,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,...,8133.83,8.3682,0.03,392,38.88,23.3739,1,4,0,0
4,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,...,8133.80,8.4294,0.03,393,38.90,23.4044,1,5,0,0


In [114]:
units = np.unique(df_pred['unit'])

In [115]:
df_pred.loc[df_pred['unit'] == -1]

,setting1,setting2,setting3,meas01,meas02,meas03,meas04,meas05,meas06,meas07,...,meas14,meas15,meas16,meas17,meas20,meas21,unit,time_cycles,labels,ground_truth


In [116]:
# Assign outliers for the last 'n' time cycles for each of the unit

units = np.unique(df_pred['unit'])
# print('Number of unique units:', len(units))
for current_unit in units:
    df_sub = df_pred.loc[df_pred['unit'] == current_unit]
    max_timecycle = max(df_sub['time_cycles'])    
    outlier_time_threshold = max_timecycle - timecycles_for_failure
    
    # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)
    
    df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1

/Users/talat/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [117]:
print(df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].loc[df_pred['unit'] == 1])

     unit  time_cycles  labels  ground_truth
0       1            1       0             0
1       1            2       0             0
2       1            3       0             0
3       1            4       0             0
4       1            5       0             0
5       1            6       0             0
6       1            7       0             0
7       1            8       0             0
8       1            9       0             0
9       1           10       0             0
10      1           11       0             0
11      1           12       0             0
12      1           13       0             0
13      1           14       0             0
14      1           15       0             0
15      1           16       0             0
16      1           17       0             0
17      1           18       0             0
18      1           19       0             0
19      1           20       0             0
20      1           21       0             0
21      1 

In [118]:
print('Number of outliers in the predicted values', len(df_pred.loc[df_pred['labels'] == -1]))
print('Number of outliers in the ground truth', len(df_pred.loc[df_pred['ground_truth'] == -1]))

Number of outliers in the predicted values 689
Number of outliers in the ground truth 1000


In [119]:
# Copy into a new dataframe with only the required columns
df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()

In [120]:
# tp = 0
# fp = 1
# fn = 2
# tn = 3

def gen_vals(ground_truth, pred_label, pos_label, neg_label):
    if (ground_truth == pos_label) & (pred_label == pos_label):
        return 0
    elif (ground_truth == neg_label) & (pred_label == pos_label):
        return 1
    elif (ground_truth == pos_label) & (pred_label == neg_label):
        return 2
    elif (ground_truth == neg_label) & (pred_label == neg_label):
        return 3

### Find out the f-measure

In [121]:
pos_label = -1
neg_label = 0

df_scoring['pos'] = -1

df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)

In [128]:
from collections import Counter
counter = Counter(df_scoring['pos'])

print(counter)

accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
print('Accuracy = ', accuracy)

precision = counter[0] / (counter[0] + counter[1])
print('Precision = ', precision)

recall = counter[0] / (counter[0] + counter[2])
print('Recall = ', recall)

f1_score = 2 * (precision * recall) / (precision + recall)
print('F1 score: ', f1_score)

Counter({3: 19341, 2: 601, 0: 399, 1: 290})
Accuracy =  0.9568125636178566
Precision =  0.579100145137881
Recall =  0.399
F1 score:  0.4724689165186501


# Making it into a function

In [125]:
def find_eval_metrics(df_pred, timecycles_for_failure):

    # Make everything in labels column into 0 or -1
    df_pred['labels'] = df_pred['labels'].apply(lambda x: 0 if x >= 0 else -1)
    
    df_pred['ground_truth'] = 0
    
    # Assign outliers for the last 'n' time cycles for each of the unit
    units = np.unique(df_pred['unit'])
    # print('Number of unique units:', len(units))
    for current_unit in units:
        df_sub = df_pred.loc[df_pred['unit'] == current_unit]
        max_timecycle = max(df_sub['time_cycles'])    
        outlier_time_threshold = max_timecycle - timecycles_for_failure

        # print('Unit=', current_unit, 'outlier threshold', outlier_time_threshold)

        df_pred['ground_truth'].loc[(df_pred['unit'] == current_unit) & (df_pred['time_cycles'] > outlier_time_threshold)] = -1
        
    # Copy into a new dataframe with only the required columns
    df_scoring = df_pred[['unit', 'time_cycles', 'labels', 'ground_truth']].copy()
    
    pos_label = -1
    neg_label = 0

    df_scoring['pos'] = -1
    df_scoring['pos'] = df_scoring.apply(lambda x: gen_vals(x['ground_truth'], x['labels'], pos_label, neg_label), axis=1)
    
    counter = Counter(df_scoring['pos'])

    accuracy = (counter[0] + counter[3]) / (counter[0] + counter[1] + counter[2] + counter[3])
    # print('Accuracy = ', accuracy)
    precision = counter[0] / (counter[0] + counter[1])
    # print('Precision = ', precision)
    recall = counter[0] / (counter[0] + counter[2])
    # print('Recall = ', recall)
    f1_score = 2 * (precision * recall) / (precision + recall)
    # print('F1 score: ', f1_score)
    
    return accuracy, precision, recall, f1_score

In [130]:
import warnings
warnings.filterwarnings("ignore")

max_timecycles_for_failure = 30

print('| timecycles_for_failure|', 'accuracy', '|', 'precision', '|', 'recall', '|', 'f1_score', '|')
print('|--|--|--|--|--|')

for i in range(1, max_timecycles_for_failure+1):
    accuracy, precision, recall, f1_score = find_eval_metrics(df_pred, i)
    print('|', i, '|', accuracy, '|', precision, '|', recall, '|', f1_score, '|')

| timecycles_for_failure| accuracy | precision | recall | f1_score |
|--|--|--|--|--|
| 1 | 0.9675730696524647 | 0.08708272859216255 | 0.6 | 0.1520912547528517 |
| 2 | 0.9667975376860065 | 0.14804063860667635 | 0.51 | 0.22947131608548932 |
| 3 | 0.9660220057195482 | 0.20899854862119013 | 0.48 | 0.2912032355915065 |
| 4 | 0.9654403567447045 | 0.2728592162554427 | 0.47 | 0.34527089072543626 |
| 5 | 0.964567883282439 | 0.33236574746008707 | 0.458 | 0.385197645079899 |
| 6 | 0.9634045853327516 | 0.3875181422351234 | 0.445 | 0.4142746314972847 |
| 7 | 0.9620474043914498 | 0.4397677793904209 | 0.4328571428571429 | 0.43628509719222464 |
| 8 | 0.9603024574669187 | 0.4862119013062409 | 0.41875 | 0.4499664204163869 |
| 9 | 0.958654452038195 | 0.5341074020319303 | 0.4088888888888889 | 0.4631843926998112 |
| 10 | 0.9568125636178566 | 0.579100145137881 | 0.399 | 0.4724689165186501 |
| 11 | 0.9544859677184819 | 0.6168359941944848 | 0.38636363636363635 | 0.475125768585802 |
| 12 | 0.9520624303232998 